In [1]:
import numpy as np
import pandas as pd

In [2]:
titanic = pd.read_csv('titanic3.csv')
titanic

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1.0,1.0,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1.0,1.0,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1.0,0.0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1.0,0.0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1.0,0.0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305,3.0,0.0,"Zabour, Miss. Thamine",female,NaN,1.0,0.0,2665,14.4542,NaN,C,NaN,NaN,NaN
1306,3.0,0.0,"Zakarian, Mr. Mapriededer",male,26.5000,0.0,0.0,2656,7.2250,NaN,C,NaN,304.0,NaN
1307,3.0,0.0,"Zakarian, Mr. Ortin",male,27.0000,0.0,0.0,2670,7.2250,NaN,C,NaN,NaN,NaN
1308,3.0,0.0,"Zimmerman, Mr. Leo",male,29.0000,0.0,0.0,315082,7.8750,NaN,S,NaN,NaN,NaN


In [5]:
titanic.groupby('sex')[['survived']].mean()

,survived
sex,
female,0.727468
male,0.190985


In [17]:
titanic.groupby('sex')['survived'].value_counts()

sex     survived
female  1.0         339
        0.0         127
male    0.0         682
        1.0         161
Name: survived, dtype: int64

In [12]:
titanic.groupby('sex')['survived'].size()

sex
female    466
male      843
Name: survived, dtype: int64

#### using pivot table

In [14]:
titanic.pivot_table('survived', index='sex', columns='pclass')

pclass,1.0,2.0,3.0
sex,,,
female,0.965278,0.886792,0.490741
male,0.340782,0.146199,0.152130


In [15]:
titanic.pivot_table('survived', index='sex', columns='pclass', aggfunc='count')

pclass,1.0,2.0,3.0
sex,,,
female,144,106,216
male,179,171,493


#### Multi-level pivot tables
Just as in the GroupBy, the grouping in pivot tables can be specified with multiple levels, and via a number of options. For example, we might be interested in looking at age as a third dimension. We'll bin the age using the pd.cut function:

In [21]:
age = pd.cut(titanic['age'], [0, 18 , 80])
titanic.pivot_table('survived', ['sex', age], 'pclass')

pclass                1.0       2.0       3.0
sex    age                                   
female (0, 18]   0.923077  0.952381  0.534483
       (18, 80]  0.966667  0.878049  0.436170
male   (0, 18]   0.750000  0.523810  0.208333
       (18, 80]  0.328671  0.087591  0.158845

In [27]:
age = pd.cut(titanic['age'], [0, 18 , 80])
titanic.pivot_table('survived', ['sex', age], 'pclass', aggfunc='count')

pclass           1.0  2.0  3.0
sex    age                    
female (0, 18]    13   21   58
       (18, 80]  120   82   94
male   (0, 18]     8   21   72
       (18, 80]  143  137  277

In [25]:
age = pd.cut(titanic['age'], [0, 10 ,20 ,30 , 40 ,50 ,60, 70 , 80])
titanic.pivot_table('survived', ['sex', age], 'pclass', aggfunc='count')

pclass           1.0  2.0  3.0
sex    age                    
female (0, 10]     1   11   29
       (10, 20]   15   16   37
       (20, 30]   33   40   53
       (30, 40]   34   20   21
       (40, 50]   23   12   11
       (50, 60]   21    4    0
       (60, 70]    5    0    1
       (70, 80]    1    0    0
male   (0, 10]     3   11   31
       (10, 20]    7   17   70
       (20, 30]   30   61  144
       (30, 40]   32   35   68
       (40, 50]   42   18   26
       (50, 60]   22   10    5
       (60, 70]   12    6    3
       (70, 80]    3    0    2

We can apply the same strategy when working with the columns as well; let's add info on the fare paid using pd.qcut to automatically compute quantiles

In [23]:
#qcut
fare = pd.qcut(titanic['fare'], 2)
titanic.pivot_table('survived', ['sex', age], [fare, 'pclass'])

fare            (-0.001, 14.454]                     (14.454, 512.329]  \
pclass                       1.0       2.0       3.0               1.0   
sex    age                                                               
female (0, 18]               NaN  0.750000  0.666667          0.923077   
       (18, 80]              NaN  0.866667  0.437500          0.966667   
male   (0, 18]               NaN  0.000000  0.264706          0.750000   
       (18, 80]              0.2  0.119048  0.154167          0.333333   

fare                                 
pclass                2.0       3.0  
sex    age                           
female (0, 18]   1.000000  0.392857  
       (18, 80]  0.884615  0.433333  
male   (0, 18]   0.687500  0.157895  
       (18, 80]  0.037736  0.194444

In [28]:
fare = pd.qcut(titanic['fare'], 2)
titanic.pivot_table('survived', ['sex', age], [fare, 'pclass'] , aggfunc='count')

fare            (-0.001, 14.454]          (14.454, 512.329]        
pclass                       1.0 2.0  3.0               1.0 2.0 3.0
sex    age                                                         
female (0, 18]                 0   4   30                13  17  28
       (18, 80]                0  30   64               120  52  30
male   (0, 18]                 0   5   34                 8  16  38
       (18, 80]                5  84  240               138  53  36

In [ ]:
#question for weather

In [29]:
dw = pd.read_csv('Dummy Weather.csv')
dw

,date,city,temperature,humidity
0,05-01-2017,New York,65,56
1,06-02-2017,New York,66,58
2,07-03-2017,New York,68,60
3,05-01-2017,Mumbai,72,80
4,06-02-2017,Mumbai,75,83
5,07-03-2017,Mumbai,73,85
6,05-01-2017,Beijing,80,26
7,06-02-2017,Beijing,78,30
8,07-03-2017,Beijing,76,35


In [41]:
#date = pd.qcut(dw['date'], 3)
dw.pivot_table(['humidity','temperature',] ,['city',],['date',] )

humidity                       temperature                      
date     05-01-2017 06-02-2017 07-03-2017  05-01-2017 06-02-2017 07-03-2017
city                                                                       
Beijing          26         30         35          80         78         76
Mumbai           80         83         85          72         75         73
New York         56         58         60          65         66         68

In [43]:
dw.pivot_table(index='city',columns='date')

humidity                       temperature                      
date     05-01-2017 06-02-2017 07-03-2017  05-01-2017 06-02-2017 07-03-2017
city                                                                       
Beijing          26         30         35          80         78         76
Mumbai           80         83         85          72         75         73
New York         56         58         60          65         66         68

In [44]:
#question for dimonds dataset

In [74]:
dia = pd.read_csv('diamonds.csv')
dia

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...,...
53935,53936,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,53937,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,53938,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,53939,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


In [76]:
dia['price'].describe()

count    53940.000000
mean      3932.799722
std       3989.439738
min        326.000000
25%        950.000000
50%       2401.000000
75%       5324.250000
max      18823.000000
Name: price, dtype: float64

In [90]:
dia.pivot_table(index='cut',columns='clarity')

Unnamed: 0                                            \
clarity              I1            IF           SI1           SI2   
cut                                                                 
Fair       29203.304762  35184.333333  22486.129902  19872.175966   
Good       23438.104167  29829.690141  25397.687179  19992.657724   
Ideal      14544.102740  34950.230198  26438.153199  23257.839107   
Premium    21809.292683  31618.252174  24706.006434  20888.881655   
Very Good  19468.869048  30083.776119  25911.145062  21119.188571   

                                                                      carat  \
clarity             VS1           VS2          VVS1          VVS2        I1   
cut                                                                           
Fair       28326.770588  25758.919540  23987.117647  29662.434783  1.361000   
Good       26270.157407  25257.855828  30087.935484  30153.115385  1.203021   
Ideal      29263.402898  29751.096825  33019.883732  32388.513814  1.222671   
Premium    27501.031674  26670.731606  33156.011364  30720.452874  1.287024   
Very Good  26296.888451  26760.945967  30937.892269  29864.704453  1.281905   

                     ...         y                   z                      \
clarity          IF  ...      VVS1      VVS2        I1        IF       SI1   
cut                  ...                                                     
Fair       0.474444  ...  5.508824  5.464638  4.386905  3.033333  3.895711   
Good       0.616338  ...  4.950806  5.269406  4.065625  3.252958  3.635064   
Ideal      0.455041  ...  4.978012  5.231869  4.158493  2.982211  3.570801   
Premium    0.603478  ...  5.018847  5.367126  4.130683  3.191478  3.692162   
Very Good  0.618769  ...  4.927503  5.115911  4.197381  3.259813  3.643451   

                                                             
clarity         SI2       VS1       VS2      VVS1      VVS2  
cut                                                          
Fair       4.211867  3.709882  3.780230  3.327647  3.444058  
Good       3.895245  3.499954  3.658231  3.068602  3.273497  
Ideal      3.865889  3.366615  3.352102  3.064304  3.221550  
Premium    4.001967  3.509703  3.565639  3.080000  3.295920  
Very Good  3.944495  3.467177  3.576144  3.031420  3.144348  

[5 rows x 64 columns]

In [94]:
price = pd.cut(dia['price'],[0,2000,2500, 3000])

In [95]:
dia.pivot_table('price', ['cut','clarity'], [price,'color'] , aggfunc='count')

price             (0, 2000]                              (2000, 2500]      \
color                     D    E    F    G    H    I   J            D   E   
cut       clarity                                                           
Fair      I1              0    4   20   27   18    7   5            1   2   
          IF              2    0    1    2    0    0   0            1   0   
          SI1             9   21   26   16   11    5   5           10   7   
          SI2             8   14   14    6    5    2   3            6   3   
          VS1             2    7    8   15    7    5   4            0   2   
          VS2             5   18   18    7    9    9   9            2   3   
          VVS1            2    1    3    1    0    0   1            0   0   
          VVS2            5    6    4    8    2    1   0            0   2   
Good      I1              2    2    9    5    3    1   1            0   3   
          IF              1    8    9   12    1    4   3            1   0   
          SI1           116  170  107   66   71   57  22            7  18   
          SI2            73   62   35   32   27   12   7           16  27   
          VS1            21   37   69   62   36   32  26            0   4   
          VS2            49   68   76   67   45   20  24            2  10   
          VVS1            7   33   26   27   24   11   0            1   3   
          VVS2           15   30   22   48   25   12   6            2   3   
Ideal     I1              6    4    5    2    0    0   0            0   0   
          IF              4   65  218  377  169   83  20            4   0   
          SI1           476  407  254  299  263  144  69           13  22   
          SI2           159  135  123  143   72   41  17           15  38   
          VS1           220  406  324  507  266  174  82           34  59   
          VS2           701  860  555  443  271  160  76           55  31   
          VVS1           93  268  312  404  249  130  23            5   3   
          VVS2          164  343  249  439  202  101  25           18  39   
Premium   I1              0    7    8   12    8    3   2            0   2   
          IF              4   16   21   58   27   17   4            0   0   
          SI1           304  301  196  213  140   76  46           13  25   
          SI2           137  147   95   91   55   36   9           20  52   
          VS1            50  164  131  303  164   81  50           13  12   
          VS2           225  390  316  316  132   62  55           12  15   
          VVS1           26   77   46  124   96   67   7            0   3   
          VVS2           54   79   65  150   81   48   6            7   9   
Very Good I1              1    4    3    4    0    1   0            1   2   
          IF              2   22   37   42   19    9   7            0   0   
          SI1           234  283  194  197  147   87  57           11  27   
          SI2            89  122   73   63   38   30  15            9  46   
          VS1            93  159  142  227  102   58  44           12  15   
          VS2           182  269  203  182  125   63  60            8  12   
          VVS1           33  131  119  131   85   43   8            0   4   
          VVS2           91  211  134  168   97   35   8           10  33   

price                   ...             (2500, 3000]                         
color                F  ...   H   I   J            D   E   F   G   H   I  J  
cut       clarity       ...                                                  
Fair      I1         3  ...   1  11   2            0   2   2   3   6   1  0  
          IF         1  ...   0   0   0            0   0   1   0   0   0  0  
          SI1        8  ...   8   1   2            2   2   5   5   2   8  3  
          SI2       12  ...   6   2   1            4  12   8   8  16   6  5  
          VS1        5  ...   6   7   1            1   1   3   5   1   1  3  
          VS2        3  ...   3   4   0            3   6   7   3